<h3><center> Next-Gen Rule Engine <font color='blue'> [Template] </center>

In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots as make_subplots


import warnings
warnings.simplefilter(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from data_basic_py import baseline_performance, data_cleaning
from feature_selection_py import feature_selection
from rules_py import rules_searching, rules_evalution

### 0. Configuration

- Note: This section needs to be updated for each project

In [ ]:
data_name = ''
cleaned_data_saving = True

Dictionary_folder_name = 'Dictionary'
save_folder_name ='Results/'
specific_project_name ='My_Project_' 

Bureau ='Equifx' # 'Experian','TransUnion', "Infinian", "LexisNexis"
preffix = ''

if Bureau =='Equifax':
    data_dictionary = pd.read_csv(Dictionary_folder_name +'Equifax_dic.csv')
elif Bureau =='Experian': 
    data_dictionary = pd.read_csv(Dictionary_folder_name +'Experian_dic.csv')
elif Bureau =='TransUnion': 
    data_dictionary = pd.read_csv(Dictionary_folder_name +'TransUnion_dic.csv')
elif Bureau =='Infinian': 
    data_dictionary = pd.read_csv(Dictionary_folder_name +'Infinian_dic.csv')
elif Bureau =='LexisNexis': 
    data_dictionary = pd.read_csv(Dictionary_folder_name +'LexisNexis_dic.csv')
else:
    data_dictionary = None


bad_flag = ''
bad_bal = ''
total_bal = ''
date_column = ''


bureau_letter_replace, num_missing_impute, cat_missing_impute = -999, -999, 'Missing'
missing_drop_threshold = 0.999

iv_threshold, iv_drop = 0.02, True
correlation_threshold, correlation_drop = 0.9, False   



### Data Importation & Cleaning

In [ ]:
data = pd.read_csv(data_name, encoding_erros ='replace')

print(data.shape)
data.head(5)

#### useless columns & months drop

In [ ]:
useless_columns = ['','','','']
useless_months = ['','','']



data = data.drop(columns = useless_columns)
data = data[~data[date_column].isin(useless_months)]

print(data.shape)
data.head(10)

#### bad flag and balance cleaning & baseline overview

In [ ]:
data_cleaning.bad_flag_cleaning(data, bad_flag, bad_bal)

print('Baseline Overview:')
baseline_performance.base_overview(data, bad_flag, bad_bal)

#### [optional] Bureau letter cleaning

In [ ]:
if Bureau == 'Equifax':
    data = data.cleaning.Equifax_Bureau_cleaning(data, Bureau, preffix, breau_letter_replace)
    num_list, cat_list = data_cleaning.num_cat_list(data, bad_flag, bad_bal, total_bal)
    print(f'Number of numerical features: {len(num_list)}. Number of categorical features: {len(cat_list}.')
    data.head(5)

#### Missing imputation

In [ ]:
data.isna().mean().sort_values(ascending=False).head(5)

In [ ]:
data = data.loc[:, data.isna().mean() > missing_drop_threshold]
data.isna().sort_values(ascending=False).head(5)

In [ ]:
num_list, cat_list = data_cleaning.num_cat_list(data, bad_flag, bad_bal, total_bal)
print(f'Number of numerical features: {len(num_list)}. Number of categorical features: {len(cat_list}.')

In [ ]:
data = data_cleaning.missing_imputation(data, num_list, cat_list, num_missing_impute, cat_missing_impute)
data.isna().sort_values(ascending=False).head(5)

In [ ]:
if cleaned_data_saving == True:
    data.to_csv('data_cleaned_version.csv')

## Part A -  Feature Prepration & Selection

In [ ]:
if cleaned_data_saving == True:
    data = pd.read_csv('data_cleaned_version.csv')
else:
    data = pd.read_csv(data_name)

num_list, cat_list = data_cleaning.num_cat_list(data, bad_flag, bad_bal, total_bal)

In [ ]:
original_num_number, original_cat_number = len(num_list), len(cat_list)
original_total_number = original_num_number + original_cat_number

#### a. Information Value

In [ ]:
iv_table = feature_selection.information_value_calculation_dictionary(data, save_folder_name + '/' + specific_project_name = 'iv_table', bad_flag, num_list, cat_list, Bureau, data_dictionary)
iv_table.head(10)

In [ ]:
feature_selection.plot_iv(iv_table, 15)

In [ ]:
iv_sur_num, iv_sur_cat, iv_sur_total = feature_selection.iv_survive(iv_table, iv_threshold, num_list, cat_list)

print(f'feature selection after information value (numerical & categorical & total): {iv_sur_num, iv_sur_cat, iv_sur_total}')

#### b. Correlation analysis

In [ ]:
corr_matrix = feature_selection.calculate_correlation(data, num_list, save_folder_name + '/' + specific_project_name = 'Correlation_Matrix')

In [ ]:
feature_selection.plot_topiv_correlation(iv_table, corr_matrix, top_x = 15, figsize = (8,6), print_num = True)

In [ ]:
highly_correlated_pairs = feature_selection.get_highly_correlated(corr_matrix, correlation_threshold, save_folder_name + '/' + specific_project_name = 'Highly_correlated_pairs')

In [ ]:
iv_sur_corr, iv_sur_corr, iv_sur_corr = feature_selection.corr_survive(iv_table, highly_correlated_pairs, iv_threshold, num_list, cat_list)

print(f'feature selection after correlation analysis (numerical & categorical & total): {iv_sur_corr, iv_sur_corr, iv_sur_corr}')

#### c. Drop features based on IV & correlations

In [ ]:
print('According to the setting:')
print(f'Drop feature based on IV is {iv_drop} with threshold {iv_threshold}')
print(f'Drop feature based on correlation is {correlation_drop} with threshold {correlation_threshold}')

if iv_drop is False and correlation_drop is False:
    num_list, cat_list = num_list, cat_list
else:
    num_list, cat_list = feature_selection.iv_correlation_filters(data, iv_table, iv_drop, iv_threshold, correlation_drop, highly_correlated_pairs, bad_flag, bad_bal, total_bal, date_column)
       
num_iv_corr, cat_iv_corr, total_iv_corr = len(num_list), len(cat_list), len(num_list) + len(cat_list)

In [ ]:
# getting the updated IV table if we decide to drop the highly correlated features

if correlation_drop == True:
    iv_table_updated = feature_selection.information_value_calculation_dictionary(data, save_folder_name + '/' + specific_project_name = 'iv_table_updated', bad_flag, num_list, cat_list, Bureau, data_dictionary)
    display(iv_table_updated.head(5)
    feature_selection.plot_iv(iv_table_updated, 10)

#### d. Automatic Logic Checks

In [ ]:
logical_num_feature_list = []

for col in num_list:
    num_feature = feature_selection.num_feature_selection_from_dis(data, col, bad_flag, bad_bal, total_bal, Bureau, data_dictionary)
    logical_num_feature_list.append(num_feature)

num_sur_logic_list = []
for col in logical_num_feature_list:
    if col != None:
        num_sur_logic_list.append(col)

In [ ]:
cat_sur_logic_list = cat_list
total_sur_logic_list = num_sur_logic_list + cat_sur_logic_list
total_sur_logic, num_sur_logic, cat_sur_logic = len(num_sur_logic_list) + len(cat_sur_logic_list), len(num_sur_logic_list), len(cat_sur_logic_list)

print(f'Before Logic checks, there are {len(num_list)} numerical features. After performing the Logic check, {num_sur_logic} numerical features are survived.')
print(f'In total, {total_sur_logic} are survived after logic check.')


<code style='badground:lightgreen; color:black'> get the updated IV table after Logic check </code>

In [ ]:
iv_table_after_logic_check = feature_selection.information_value_calculation_dictionary(data, save_folder_name + '/' + specific_project_name = 'iv_table_after_logic_check', bad_flag, num_sur_logic_list, cat_sur_logic_list, Bureau, data_dictionary)
iv_table.head(10)

#### e. Decile Analysis

In [ ]:
# for numerical ones
for col in iv_table_after_logic_check['Variable'][:10]:
    if col in num_sur_logic_list:
        feature_selection.distribution_plot_based_on_type_both(data, col, bad_flag, total_bal, bad_bal, Bureau, data_dictionary)
        print('')

In [ ]:
# for categorical ones
for col in iv_table_after_logic_check['Variable'][:10]:
    if col in cat_sur_logic_list:
        feature_selection.distribution_plot_based_on_type_both(data, col, bad_flag, total_bal, bad_bal, Bureau, data_dictionary)
        print('')

In [ ]:
# for any data type
for col in iv_table_after_logic_check['Variable'][:10]:
    feature_selection.distribution_plot_based_on_type_both(data, col, bad_flag, total_bal, bad_bal, Bureau, data_dictionary)
    print('')

#### f. Distribution Analysis (Good vs. Bad)

In [ ]:
for col in iv_table_after_logic_check['Variable'][:10]:
    if col in cat_sur_logic_list:
        feature_selection.good_bad_distribution_definiotion(data, col, bad_flag, Bureau, data_dictionary, narrow=True, bins = 15)
    

#### g. Funnel plot for feature selection process

In [ ]:
expert_selection = 15
feature_selection.funnel_feature_selection(original_total_number, iv_sur_total, total_sur_corr, total_iv_corr, expert_selection)

#### Expoert the results of Feature selectio process

In [ ]:
with pd.ExcelWriter(save_folder_name +'/' + specific_project_name +'Feature_Selection_results.xlsx', engine='xlsxwriter') as writer:
    iv_table.to_excel(writer, sheet_name ='IV')
    corr_matrix.to_excel(writer, sheet_name ='Corr_Matrix')
    highly_correlated_pairs.to_excel(writer, sheet_name ='Highly_corr_pairs')
    if correlation_drop == True:
        iv_table_updated.to_excel(writer, sheet_name ='IV_after_Corr')
    iv_table_after_logic_check.to_excel(writer, sheet_name='IV_after_logic_check')

## Part B - Rules Construction & Evaluation

**we are aiming to get a dataset with format as below**

    Variable | Valid Min | Valid Max | Search Min | Search Max | Step | Direction | Type |

In [ ]:
variable_dateframe  = rule_searching.final_feature_for_rule_construction()

variable_dateframe.loc([len(variable_dateframe)] = ['Variable 1', 0, 9999, 900, 1000, 10, -1, 'Num']
variable_dateframe.loc([len(variable_dateframe)] = ['Variable 2', -999, 999, -30, 20, 5, 1, 'Num']
variable_dateframe.loc([len(variable_dateframe)] = ['Variable 3', 'N/A', 'N/A', ['Cat 1','Cat2'], 'N/A', 'N/A', 'N/A', 'Cat']

display(variable_dateframe)

In [ ]:
variable_dateframe.value_counts()

**Baseline Overview**

In [ ]:
baseline_performance.base_overview(data, bad_flag, total_bal, bad_bal)

**<font color='green'> Notice:**

To ensure the effectiveness of the rules, we could apply several restrictions to output eligible rules only.

### Rules Generation

In [ ]:
perfered_metric ='GB'# or 'Bal BR'

if perfered_metric =='GB':
    selected_function, Metric_name = rules_searching.combine_checking_gb_ratio, 'G:B ratio'
elif perfered_metric =='Bal BR':
    selected_function, Metric_name = rules_searching.combine_checking_badbal_times, 'Bad Balance Times'   
else:
    pritn('Error!')

print(f'The selected Rule Performance Evaluation Metric is {perfered_metric})

In [ ]:
min_bads_pct_detect = 0.05
total_bads = len(data[data[bad_flag]==1)
min_bads = np.round(min_bads_pct_detect * total_bads,0)

print(f'Proposed rule should at least detect {min_bads} real defaulters, which counts  {min_bads_pct_detect*100}% of total defaulters.')

#### one-dimensional Rules

In [ ]:
rules_1D = rules_searching.Rules_Optimisation_Search_Algorithm_1D(data, variable_dateframe, bad_flag, bad_bal, total_bal, selected_function, Metric_name, min_bads)
rules_1D.to_csv(save_folder_name + '/' + specific_project_name +'1D_Rules.csv')

In [ ]:
print(f'In total, we obtained {len(rules_1D} eligible one-dimensional rules. Based on metric {perfered_metric} and with minimum {min_bads} ({min_bads_pct_detect*100}%) defaulters removed.')
rules_1D.head(5)

#### two-dimensional Rules

In [ ]:
rules_2D = rules_searching.Rules_Optimisation_Search_Algorithm_2D(data, variable_dateframe, bad_flag, bad_bal, total_bal, selected_function, Metric_name, min_bads)
rules_2D.to_csv(save_folder_name + '/' + specific_project_name +'2D_Rules.csv')

In [ ]:
print(f'In total, we obtained {len(rules_2D} eligible one-dimensional rules. Based on metric {perfered_metric} and with minimum {min_bads} ({min_bads_pct_detect*100}%) defaulters removed.')
rules_2D.head(5)

#### three-dimensional Rules

when utilsing **OR** condition:
- A or B and C means (A or B) and C
- A and B or C means A and (B or C) 

In [ ]:
rules_3D = rules_searching.Rules_Optimisation_Search_Algorithm_3D(data, variable_dateframe, bad_flag, bad_bal, total_bal, selected_function, Metric_name, min_bads)
rules_3D.to_csv(save_folder_name + '/' + specific_project_name +'3D_Rules.csv')

In [ ]:
print(f'In total, we obtained {len(rules_3D} eligible one-dimensional rules. Based on metric {perfered_metric} and with minimum {min_bads} ({min_bads_pct_detect*100}%) defaulters removed.')
rules_3D.head(5)

#### Export Rules

In [ ]:
with pd.ExcelWriter(save_folder_name +'/' + specific_project_name +'All_Rules.xlsx', engine='xlsxwriter') as writer:
    rules_1D.to_excel(writer, sheet_name ='1D_rules')
    rules_2D.to_excel(writer, sheet_name ='2D_rules')
    rules_3D.to_excel(writer, sheet_name ='3D_rules')

### Rules Performance Deep Dive

In [ ]:
rule_1 = (data['Variable 1'] < 950) & (data['Variable 2'] > 0)
rule_2 = (data['Variable 1'] < 1000) & (data['Variable 2'] > -5) & (data['Variable 3'].isin('Cat 1'))

In [ ]:
rules_searching.rule_checking(data, data[rule1], bad_flag, bad_bal, total_bal)

rules_searching.rule_checking(data, data[rule2], bad_flag, bad_bal, total_bal)

rules_searching.rule_checking(data, data[rule1 & rule2], bad_flag, bad_bal, total_bal)

rules_searching.rule_checking(data, data[rule1 | rule2], bad_flag, bad_bal, total_bal)

**could include another rule/threshold to benchmark**

In [ ]:
benchmark_rule = (data['benchmark_variable'] < 500)
rules_searching.rule_checking(data, data[benchmark_rule], bad_flag, bad_bal, total_bal)

#### Redundancy Analysis

In [ ]:
rule1_total, rule1_bad = 100,50
rule2_total, rule2_bad = 80, 35
both_total, both_bad = 130, 75

rules_evalute.Two_rules_redundancy(rule1_total, rule1_bad, rule2_total, rule2_bad ,both_total, both_bad)

In [ ]:
rule1_total, rule1_bad = 100,50
rule2_total, rule2_bad = 80, 35
rule3_total, rule3_bad = 30, 12
rule12_both_total, rule12_both_bad = 130, 75
rule13_both_total, rule13_both_bad = 112, 60
rule23_both_total, rule23_both_bad = 98, 43

rules_evalute.Three_rules_redundancy(rule1_total, rule1_bad, rule2_total, rule2_bad, rule3_total, rule3_bad, rule12_both_total, rule12_both_bad, rule13_both_total, rule13_both_bad, rule23_both_total, rule23_both_bad)


#### Segment-level analysis

In [ ]:
selected_segment ='channel'

In [ ]:
# for a single rule or multiple rules implemented together
group_results_two = rules_evalute.group_analysis_table_2rules(data, (rule_1 | rule_2), benchmark_rule, selected_segment, bad_flag, bad_bal, total_bal)
rule_evaluate.rule_performance_two(group_results_two, top_x = 15, 
                                   original_bal = 'original_Total_Bal', new_bal ='new_Total_Bal',
                                   original_bad_bal = 'original_Bad_Bal', new_bal ='new_Bad_Bal',
                                   rule1_name ='ML_rules', rule2_name='Benchmark',
                                  )                      

In [ ]:
# for multiple rules, show performance separately - for benchmarking
group_results_one = rules_evalute.group_analysis_table_1rule(data, (rule_1 | rule_2), selected_segment, bad_flag, bad_bal, total_bal)
rule_evaluate.rule_performance_one(group_results_one, top_x = 15, 
                                   original_bal = 'original_Total_Bal', new_bal ='new_Total_Bal',
                                   original_bad_bal = 'original_Bad_Bal', new_bal ='new_Bad_Bal',
                                  )  